# Day 20 - o1

In [1]:
#!/usr/bin/env python3

from collections import deque

def solve():
    # Read the grid from input.txt
    with open("input.txt", "r") as f:
        grid = [line.rstrip("\n") for line in f]
    
    rows = len(grid)
    cols = len(grid[0])
    
    # Find S and E
    start = None
    end = None
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 'S':
                start = (r, c)
            elif grid[r][c] == 'E':
                end = (r, c)
    
    # --- A helper to check if a cell is in bounds and not a wall (normal BFS).
    def is_valid(r, c):
        return 0 <= r < rows and 0 <= c < cols and grid[r][c] != '#'
    
    # --- BFS to compute distance from a single source to all track cells (including S, E).
    #     Return a dist array where dist[(r, c)] = distance from source to (r, c) or None if unreachable.
    def bfs_dist_from(source):
        dist = {}
        queue = deque()
        dist[source] = 0
        queue.append(source)
        
        while queue:
            rr, cc = queue.popleft()
            for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                nr, nc = rr + dr, cc + dc
                if is_valid(nr, nc) and (nr, nc) not in dist:
                    dist[(nr, nc)] = dist[(rr, cc)] + 1
                    queue.append((nr, nc))
        return dist
    
    # 1) BFS from S to get distance to all track cells, including dist(S, T).
    dist_from_S = bfs_dist_from(start)
    
    # 2) BFS from E to get distance to all track cells, including dist(E, T).
    dist_from_E = bfs_dist_from(end)
    
    # 3) The normal shortest distance from S to E
    #    If unreachable, there's no sense in proceeding; but presumably it's reachable.
    if end not in dist_from_S:
        # No path from S to E without cheating, so time saved can be anything...
        # The puzzle presumably always has a path. But handle the corner case:
        print(0)
        return
    distSE = dist_from_S[end]
    
    # --- Now, for each track cell T, find all T' reachable in <=2 steps ignoring walls.
    #     We'll do a BFS/DFS ignoring walls with a limit of 2 steps.
    
    # A helper to check in-bounds (ignoring if it's a wall).
    def in_bounds(r, c):
        return 0 <= r < rows and 0 <= c < cols
    
    # Precompute a list of all track cells to speed up iteration.
    track_cells = []
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] != '#':  # includes ., S, E
                track_cells.append((r, c))
    
    # For quick membership check:
    track_set = set(track_cells)
    
    # We'll map each track cell T -> list of (T', steps_ignoring_walls)
    # where steps_ignoring_walls <= 2 and T' is track.
    ignore_wall_reach = {}
    
    for (r, c) in track_cells:
        # BFS/DFS up to depth 2 ignoring walls
        visited = set()
        q = deque()
        visited.add((r, c))
        q.append((r, c, 0))  # (row, col, distance)
        results = []
        
        while q:
            rr, cc, d = q.popleft()
            # If this cell is track, we can record it
            if (rr, cc) in track_set and d > 0 and d <= 2:
                # record that we can reach (rr, cc) in d steps ignoring walls
                results.append((rr, cc, d))
            
            if d < 2:
                # We can keep going
                for dr, dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                    nr, nc = rr + dr, cc + dc
                    if in_bounds(nr, nc) and (nr, nc) not in visited:
                        visited.add((nr, nc))
                        q.append((nr, nc, d+1))
        
        ignore_wall_reach[(r, c)] = results
    
    # Now we have:
    # dist_from_S[T] = distance from S to T in normal BFS
    # dist_from_E[T] = distance from E to T in normal BFS
    # ignore_wall_reach[T] = list of (T', k) with k in [1,2]
    
    # Finally, find all cheats that save >= 100 picoseconds
    count_big_saves = 0
    
    for T in track_cells:
        # We only care if T is reachable from S at all:
        if T not in dist_from_S:
            continue
        distS_T = dist_from_S[T]
        
        for (Tprime_r, Tprime_c, k) in ignore_wall_reach[T]:
            Tprime = (Tprime_r, Tprime_c)
            # Also T' must be reachable to E
            if Tprime not in dist_from_E:
                continue
            
            cheat_time = distS_T + k + dist_from_E[Tprime]
            saved = distSE - cheat_time
            if saved >= 100:
                count_big_saves += 1
    
    # count_big_saves is the number of (T, T') pairs whose cheat saves >= 100 picoseconds.
    print(count_big_saves)


if __name__ == "__main__":
    solve()


1402


## Part 2

In [2]:
#!/usr/bin/env python3

from collections import deque

def solve():
    # Read the grid from input.txt
    with open("input.txt", "r") as f:
        grid = [line.rstrip("\n") for line in f]
    
    rows = len(grid)
    cols = len(grid[0])

    # Locate S (start) and E (end)
    start = None
    end = None
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 'S':
                start = (r, c)
            elif grid[r][c] == 'E':
                end = (r, c)

    # A helper to check if a cell is within bounds and not a wall (for normal BFS).
    def is_valid(r, c):
        return 0 <= r < rows and 0 <= c < cols and grid[r][c] != '#'
    
    # Standard BFS to get normal distance from one source to all reachable track cells.
    # dist[(r, c)] = number of normal steps from source to (r, c), or None if unreachable.
    def bfs_dist_from(source):
        dist = {}
        queue = deque()
        dist[source] = 0
        queue.append(source)
        
        while queue:
            rr, cc = queue.popleft()
            for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nr, nc = rr + dr, cc + dc
                if is_valid(nr, nc) and (nr, nc) not in dist:
                    dist[(nr, nc)] = dist[(rr, cc)] + 1
                    queue.append((nr, nc))
        return dist

    # 1) BFS from S and E (normal moves)
    dist_from_S = bfs_dist_from(start)
    dist_from_E = bfs_dist_from(end)

    # If E isn't reachable from S normally, the "no-cheat" distance doesn't exist.
    if end not in dist_from_S:
        # Edge case: no normal path => the puzzle statement usually implies S->E is possible.
        # If truly unreachable, then 0 cheats would be definable for "saving >= 100", or
        # possibly everything is infinite. We’ll just return 0.
        print(0)
        return

    distSE = dist_from_S[end]  # The baseline (no-cheat) fastest path

    # Collect all track cells (including S, E). 
    # We'll only do ignoring-wall BFS from cells that are reachable from S anyway.
    # (Because if T isn't reachable from S in normal BFS, it can't help reduce the S->E path.)
    track_cells = [cell for cell in dist_from_S.keys()]  # all cells that S can reach

    # Also, we need to know which cells are *in-bounds* but not necessarily passable:
    # for the ignoring-wall BFS, we only require in-bounds, ignoring walls.
    def in_bounds(r, c):
        return 0 <= r < rows and 0 <= c < cols

    # For each track cell T, run a BFS/DFS ignoring walls up to 20 steps
    # to find all track cells T' we can reach in d <= 20 ignoring-wall steps.
    # We'll store the *minimum* ignoring distance for each T' (because
    # any longer path to the same T' doesn't give a new cheat).
    #
    # Then compute how much time it saves: distSE - (distS_T + d + distT'E).
    # If >= 100, count it.
    
    count_big_saves = 0

    for T in track_cells:
        distS_T = dist_from_S[T]  # normal BFS distance from S to T

        # We'll do a BFS ignoring walls from T, up to depth 20
        # and record the minimal ignoring distance to each track cell reached.
        ignore_dist = {}
        queue = deque()
        ignore_dist[T] = 0
        queue.append(T)

        while queue:
            rr, cc = queue.popleft()
            current_d = ignore_dist[(rr, cc)]
            if current_d == 20:
                # Can't go deeper than 20 ignoring-wall steps
                continue

            for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nr, nc = rr + dr, cc + dc
                if in_bounds(nr, nc):
                    # We can step here ignoring walls.
                    # If we haven't visited it yet (or found a cheaper ignoring distance),
                    # record it and queue it.
                    nd = current_d + 1
                    if (nr, nc) not in ignore_dist or nd < ignore_dist[(nr, nc)]:
                        ignore_dist[(nr, nc)] = nd
                        queue.append((nr, nc))

        # Now ignore_dist has the minimal ignoring distance from T to many in-bounds cells.
        # We only care about those that are *track cells* and also reachable from E in normal BFS.
        # Because to complete the cheat, we must end on a real track cell T' (grid[T'] != '#').
        # And also we want T' that can reach E normally (dist_from_E[T'] exists).
        for Tprime, k in ignore_dist.items():
            # Must be a valid track cell for the cheat to *end* there
            if grid[Tprime[0]][Tprime[1]] == '#':
                continue
            # Must be reachable to E in normal BFS
            if Tprime not in dist_from_E:
                continue

            # k is the ignoring-wall distance from T to Tprime (<= 20 by construction)
            # Compute total path time with this cheat
            cheat_time = distS_T + k + dist_from_E[Tprime]
            saved = distSE - cheat_time
            if saved >= 100:
                count_big_saves += 1

    print(count_big_saves)

if __name__ == "__main__":
    solve()


1020244
